In [5]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
import cv2
from PIL import Image, ImageTk
import numpy as np

from tensorflow.keras.models import model_from_json

# Initialize Tkinter window
top = tk.Tk()
top.geometry('800x600')
top.title('Emotion Detector')
top.configure(background='#CDCDCD')

# Load the pre-trained model
def FacialExpressionModel(json_file, weights_file):
    with open(json_file, "r") as file:
        loaded_model_json = file.read()
        model = model_from_json(loaded_model_json)

    model.load_weights(weights_file)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Load pre-trained model and cascade classifier
facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = FacialExpressionModel("model_a1.json", "model_weights1.h5")

# Define emotions list
EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Function to detect emotions from live video feed
def Detect():
    try:
        cap = cv2.VideoCapture(0)  # Use the default camera (usually webcam)
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = facec.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

            for (x, y, w, h) in faces:
                fc = gray_frame[y:y + h, x:x + w]
                roi = cv2.resize(fc, (48, 48))
                pred = EMOTIONS_LIST[np.argmax(model.predict(roi[np.newaxis, :, :, np.newaxis]))]
                cv2.putText(frame, pred, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            cv2.imshow('Emotion Detector', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print("Error:", e)

# Create Detect button to start emotion detection
detect_button = Button(top, text="Detect Emotion", command=Detect, padx=10, pady=5)
detect_button.configure(background="#364156", foreground='white', font=('arial', 10, 'bold'))
detect_button.place(relx=0.79, rely=0.46)

# Upload button not needed for live feed
# upload_button = Button(top, text="Upload Image", command=upload_image, padx=10, pady=5)
# upload_button.configure(background="#364156", foreground='white', font=('arial', 20, 'bold'))
# upload_button.pack(side='bottom', pady=50)

# Label for displaying emotion
label1 = Label(top, background='#CDCDCD', font=('arial', 15, 'bold'))
label1.pack(side='bottom', expand='True')

# Title label
heading = Label(top, text='Emotion Detector', pady=20, font=('arial', 25, 'bold'))
heading.configure(background='#CDCDCD', foreground="#364156")
heading.pack()

top.mainloop()

1/1 [==============================] - 0s 63ms/step
